In [ ]:
# Check the current GPU device (if available)
import tensorflow as tf
tf.test.gpu_device_name()


''

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Specify the paths to the parent folder containing "train" and "test"
parent_data_dir = "/content/drive/My Drive/finaldataset/skin"
train_data_dir = parent_data_dir + "/train"
test_data_dir = parent_data_dir + "/test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)  # Normalize pixel values for test data

# Set the batch size
batch_size = 32

# Create training and testing data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)


Found 7452 images belonging to 6 classes.
Found 1644 images belonging to 6 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2


# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),# Pooling layer
    Dropout(0.5),
    layers.Dense(128, activation='relu'),  # Additional dense layer (adjust as needed)
    layers.Dense(6, activation='softmax')  # 23 disease classes + 1 for normal
])


from tensorflow.keras.optimizers import Adam

# Change the learning rate to a different value, e.g., 0.001
optimizer = Adam(learning_rate=0.001)

# Compile the model with the updated optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()  # View the model architecture


9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                           

In [ ]:
# Set the number of training epochs
epochs = 18
# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)


Epoch 1/18
165/233 [====================>.........] - ETA: 8:58 - loss: 1.0405 - accuracy: 0.5584

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

52/52 [==============================] - 9s 170ms/step - loss: 3.5397 - accuracy: 0.4331
Test accuracy: 0.4330900311470032


In [ ]:
# done by us-- these can be edited

import pickle
# Replace 'your_model' with the actual name of your model variable
with open('/content/drive/My Drive/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('/content/drive/My Drive/model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)


your_input_data= '/content/drive/My Drive/finaldataset/skin/test/vitiligo/roi300.jpg'

class_labels =  ["Acne and Rosacea Photos","Eczema Photos","Normal","Psoriasis pictures Lichen Planus and related diseases","Tinea Ringworm Candidiasis and other Fungal Infections","vitiligo"]
# print(class_labels)
# Load and preprocess the uploaded image
img = image.load_img(your_input_data, target_size=(224, 224))  # Resize to (224, 224)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize pixel values


# Test the loaded model
result = loaded_model.predict(img_array)  # Replace 'your_input_data' with your actual input data
print(result)

# Get the predicted class index
predicted_class_index = np.argmax(result)

# Get the predicted class label using the extracted class labels
predicted_class_label = class_labels[predicted_class_index]

# Get the probability of the predicted class
predicted_probability = result[0][predicted_class_index]
print(predicted_class_label)

1/1 [==============================] - 1s 1s/step
[[9.4657531e-04 1.7740321e-11 9.9860030e-01 1.8161091e-11 1.1686753e-09
  4.5307429e-04]]
Normal


In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Load the trained model (assuming it's already loaded or defined in your Colab session)
# If not, you can load it using the code you provided earlier
# model = ...

# Upload an image file
uploaded = files.upload()

# Get the uploaded image file name (assuming only one file is uploaded)
uploaded_image_filename = list(uploaded.keys())[0]

# Define the path to the parent folder containing class labels (training data)
parent_data_dir = "/content/drive/My Drive/finaldataset/skin/train"

# Get the class labels from the subdirectories in the training data folder
class_labels =  ["Acne and Rosacea Photos","Eczema Photos","Normal","Psoriasis pictures Lichen Planus and related diseases","Tinea Ringworm Candidiasis and other Fungal Infections","vitiligo"]
# print(class_labels)
# Load and preprocess the uploaded image
img = image.load_img(uploaded_image_filename, target_size=(224, 224))  # Resize to (224, 224)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize pixel values

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class label using the extracted class labels
predicted_class_label = class_labels[predicted_class_index]

# Get the probability of the predicted class
predicted_probability = predictions[0][predicted_class_index]

# Display the results
print(f'Predicted class: {predicted_class_label}')

print(f'Probability: {predicted_probability:.2f}')



Saving chapped-fissured-feet-33.jpg to chapped-fissured-feet-33.jpg
1/1 [==============================] - 0s 22ms/step
Predicted class: Psoriasis pictures Lichen Planus and related diseases
Probability: 0.41


In [ ]:
model.save('final_model.h5')
model.save('my_model.keras')

In [ ]:
import tensorflow as tf

# Load the saved model
loaded_model = tf.keras.models.load_model("final_model.h5")

parent_data_dir = "/content/drive/My Drive/finaldataset/skin/test/Normal/0_0_aidai_0029.jpg"

# Get the class labels from the subdirectories in the training data folder
class_labels =  ["Acne and Rosacea Photos","Eczema Photos","Normal","Psoriasis pictures Lichen Planus and related diseases","Tinea Ringworm Candidiasis and other Fungal Infections","vitiligo"]
# print(class_labels)
# Load and preprocess the uploaded image
img = image.load_img(uploaded_image_filename, target_size=(224, 224))  # Resize to (224, 224)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize pixel values

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class label using the extracted class labels
predicted_class_label = class_labels[predicted_class_index]

print(f'Predicted class: {predicted_class_label}')


1/1 [==============================] - 1s 753ms/step
Predicted class: Psoriasis pictures Lichen Planus and related diseases


In [ ]:
from google.colab import files

# Download the file to your local system
files.download('final_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>